# Polygon.io

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')

True

In [2]:
from utils.restapi import get_env_licence, api_get

licence=get_env_licence('polygonio')
print(licence)

cigqqVmWxZyND94q3lYiQ5x6kszQJGLP


## [Aggregate (bars)](https://polygon.io/docs/stocks/get_v2_aggs_ticker__stocksticker__range__multiplier___timespan___from___to)
Request parameters:
- ticker(string): The ticker symbol of the stock/equity.
- multiplier(int): The size of the timespan multiplier. 
- timespan(string): The size of the time window.
- from(datetime): The start of the aggregate time window. Either a date with the format YYYY-MM-DD or a millisecond timestamp.
- to(datetime): The end of the aggregate time window. Either a date with the format YYYY-MM-DD or a millisecond timestamp.
- adjusted(bool): Whether or not the results are adjusted for splits. By default, results are adjusted. Set this to false to get results that are NOT adjusted for splits.
- sort(enum): Sort the results by timestamp. asc will return results in ascending order (oldest at the top), desc will return results in descending order (newest at the top).
- limit(int): Limits the number of base aggregates queried to create the aggregate results. Max 50000 and Default 5000. Read more about how limit is used to calculate aggregate results in our article on [Aggregate Data API Improvements](https://polygon.io/blog/aggs-api-updates/).

Response Attributes:
- ticker(string): The exchange symbol that this item is traded under.
- adjusted(boolean): Whether or not this response was adjusted for splits.
- queryCount(integer): The number of aggregates (minute or day) used to generate the response.
- request_id(string): A request id assigned by the server.
- resultsCount(integer): The total number of results for this request.
- status(string): The status of this request's response.
- results(array):
    - c(number): The close price for the symbol in the given time period.
    - h(number): The highest price for the symbol in the given time period.
    - l(number): The lowest price for the symbol in the given time period.
    - n(integer): The number of transactions in the aggregate window.
    - o(number): The open price for the symbol in the given time period.
    - otc(boolean): Whether or not this aggregate is for an OTC ticker. This field will be left off if false.
    - t(integer): The Unix Msec timestamp for the start of the aggregate window.
    - v(number): The trading volume of the symbol in the given time period.
    - vw(number): The volume weighted average price.
- next_url(string): If present, this value can be used to fetch the next page of data.

In [3]:
import dataclasses
import datetime

@dataclasses.dataclass
class PolygonIOBarParam:
    _ticker:str
    _multiplier:int
    _timespan:str
    _dfrom:datetime.date
    _dto:datetime.date
    _adjust:bool=False
    _sort:str='asc'
    _limit:int=5000
    _licence:str|None=None
    @property
    def ticker(self)->str:
        return self._ticker
    @ticker.setter
    def ticker(self,t:str)->None:
        self._ticker=t
    @property
    def multiplier(self)->int:
        return self.multiplier
    @multiplier.setter
    def multiplier(self,m:int)->None:
        self._multiplier=m
    @property
    def timespan(self)->str:
        return self._timespan
    @timespan.setter
    def timespan(self,t:str)->None:
        self._timespan=t
    @property
    def from_date(self)->str:
        return self._dfrom.isoformat()
    @from_date.setter
    def from_date(self,d:datetime.date)->None:
        self._dfrom=d
    @property
    def to_date(self)->str:
        return self._dto.isoformat()
    @to_date.setter
    def to_date(self,d:datetime.date)->None:
        self._dto=d
    @property
    def adjust(self)->bool:
        return self._adjust
    @adjust.setter
    def adjust(self,a:bool)->None:
        self._adjust=a
    @property
    def sort(self)->str:
        return self._sort
    @sort.setter
    def sort(self,s:str)->None:
        self._sort=s
    @property
    def limit(self)->int:
        return self._limit
    @limit.setter
    def limit(self,l:int)->None:
        self._limit=l
    @property
    def licence(self)->str|None:
        return self._licence
    @licence.setter
    def licence(self,l:str|None)->None:
        self._licence=l
    def url(self)->str:
        return f'https://api.polygon.io/v2/aggs/ticker/{self._ticker}/range/{self._multiplier}/{self._timespan}/{self._dfrom}/{self._dto}?adjusted={self._adjust}&sort={self._sort}&limit={self._limit}&apiKey={self._licence}'

In [4]:
param=PolygonIOBarParam('AAPL',1,'minute',datetime.date(2023,1,9),datetime.date(2023,1,9))

In [5]:
param.licence=licence
url=param.url()

In [6]:
import pandas as pd
pd.DataFrame(api_get(url)['results'].tolist()).set_index('t').rename_axis('Date')

,v,vw,o,c,h,l,n
Date,,,,,,,
1673254800000,2312.0,129.8763,129.60,129.89,130.30,129.6000,85
1673254860000,6206.0,129.7086,129.80,129.70,129.80,129.6800,74
1673254920000,2386.0,129.8947,129.84,130.00,130.00,129.8000,83
1673254980000,3372.0,130.0070,130.00,129.99,130.01,129.9900,81
1673255040000,1087.0,130.0019,129.98,129.99,129.99,129.9800,41
...,...,...,...,...,...,...,...
1673312100000,528.0,129.8164,129.81,129.80,129.84,129.8000,19
1673312160000,979.0,129.8221,129.83,129.82,129.83,129.8100,24
1673312220000,1397.0,129.8126,129.81,129.81,129.81,129.8100,25


In [7]:
from data.dataframe.schemas import KLineSchema

class PolygonIOData:
    _licence:str|None
    def __init__(self,l:str|None) -> None:
        self._licence=l
    def _get_url(self,ticker:str,punit:int,period:str,
                 dfrom:datetime.date|pd.Timestamp,dto:datetime.date|pd.Timestamp,
                 adjusted:bool=False)->str:
        param=PolygonIOBarParam(ticker,punit,period,dfrom,dto,adjusted)
        param.licence=self._licence
        return param.url()
    def _get_bars(self,url:str)->pd.DataFrame:
        return pd.DataFrame(api_get(url)['results'].tolist()).set_index('t')
    def get_bars(self,ticker:str,punit:int,period:str,
                 dfrom:datetime.date|pd.Timestamp,dto:datetime.date|pd.Timestamp)->pd.DataFrame:
        url=self._get_url(ticker,punit,period,dfrom,dto)
        data=self._get_bars(url)
        url=self._get_url(ticker,punit,period,dfrom,dto,True)
        data['Adj Close']=self._get_bars(url)['c']
        data.index=[pd.Timestamp(x,unit='ms') for x in data.index]
        return KLineSchema(data.rename(columns={'o':'Open','c':'Close','h':'High','l':'Low','v':'Volume'})
                           .rename_axis('Date'))

### Get bar data

In [8]:
pgio=PolygonIOData(licence)
data=pgio.get_bars('AAPL',1,'minute',datetime.date(2023,1,9),datetime.date(2023,1,9))

In [9]:
data.head()

,Volume,Open,Close,High,Low,Adj Close
Date,,,,,,
2023-01-09 09:00:00,2312.0,129.60,129.89,130.30,129.60,129.89
2023-01-09 09:01:00,6206.0,129.80,129.70,129.80,129.68,129.70
2023-01-09 09:02:00,2386.0,129.84,130.00,130.00,129.80,130.00
2023-01-09 09:03:00,3372.0,130.00,129.99,130.01,129.99,129.99
2023-01-09 09:04:00,1087.0,129.98,129.99,129.99,129.98,129.99
